In [1]:
import neutron_analysis.shielding as shielding
import neutron_analysis.utils as utils

import pandas as pd
import numpy as np

from io import StringIO

# Shielding

In [2]:
from importlib import reload
reload(shielding)

<module 'neutron_analysis.shielding' from '/Users/wpk/Documents/Research/CurrentResearch/Projects/CCI/collaboration_646/Nick_sharp_workflow/neutron_analysis/neutron_analysis/shielding.py'>

In [3]:
df_mass = (
    shielding.read_mass_excel('./example_input_data.xlsx', sheet_name='Sheet1', extra_index=['sample','mass'])
)
df_mass.head()

mass_percent  mass_frac
sample   mass radius length density  element                         
sample_0 0.3  0.65   0.15   1.506792 H                0.75     0.0075
                                     C                2.00     0.0200
                                     N                1.00     0.0100
                                     O               47.00     0.4700
                                     Na               1.22     0.0122

In [4]:
s = StringIO()
df_mass.to_csv(s)

In [5]:
with StringIO(s.getvalue()) as f:
    out = (
        pd.read_csv(f, index_col=list(range(0,6)))
#        .set_index(['sample','mass','radius','length','density','element'])
    )

In [6]:
S0 = shielding.NeutronShieldingPT(out)

In [7]:
# create shielding objects
S = shielding.NeutronShieldingUser(df_mass)

In [8]:
S.shielding_ratio('sample_0', value_name='values')

values              sample_0/sample  \
                                     f_scatter f0_noscatter       f_scatter   
sample   mass radius length density                                           
sample_0 0.3  0.65   0.15   1.506792  0.997731     0.995817        1.000000   
sample_1 0.4  0.65   0.15   2.009056  0.997114     0.994619        1.000619   
sample_2 0.3  0.65   0.15   1.506792  0.997653     0.995551        1.000077   

                                                     uncert               
                                     f0_noscatter f_scatter f0_noscatter  
sample   mass radius length density                                       
sample_0 0.3  0.65   0.15   1.506792     1.000000  0.000000     0.000000  
sample_1 0.4  0.65   0.15   2.009056     1.001205  0.000126     0.000245  
sample_2 0.3  0.65   0.15   1.506792     1.000267  0.000016     0.000054

In [9]:
s = StringIO()
S.shielding_ratio('sample_0', value_name='values').to_csv(s)
s

In [10]:
ss = """
,,,,,values,values,sample_0/sample,sample_0/sample,uncert,uncert
,,,,,f_scatter,f0_noscatter,f_scatter,f0_noscatter,f_scatter,f0_noscatter
sample,mass,radius,length,density,,,,,,
sample_0,0.3,0.65,0.15,1.5067923606333287,0.9977306051161332,0.9958168360313382,1.0,1.0,0.0,0.0
sample_1,0.4,0.65,0.15,2.009056480844438,0.9971138357174368,0.9946185064665436,1.0006185546490312,1.00120481325956,0.00012589752631410905,0.00024460799811984983
sample_2,0.3,0.65,0.15,1.5067923606333287,0.9976533767929456,0.9955508457237608,1.0000774099752319,1.0002671790282935,1.5764165458358606e-05,5.4295044174204355e-05
"""

In [11]:
with  StringIO(ss) as f:
    check = pd.read_csv(f, index_col=list(range(0, 5)), header=[0,1])

In [12]:
pd.testing.assert_frame_equal(check, S.shielding_ratio('sample_0',value_name='values'))

In [13]:
pd.testing.assert_frame_equal(check, S0.shielding_ratio('sample_0', value_name='values'))

In [17]:
S.shielding_ratio('sample_0') - S0.shielding_ratio('sample_0')

sample_0/sample                \
                                           f_scatter  f0_noscatter   
sample   mass radius length density                                  
sample_0 0.3  0.65   0.15   1.506792             NaN           NaN   
                            1.506792             NaN           NaN   
sample_1 0.4  0.65   0.15   2.009056   -2.160494e-08 -4.051679e-08   
sample_2 0.3  0.65   0.15   1.506792             NaN           NaN   
                            1.506792             NaN           NaN   

                                            uncert                
                                         f_scatter  f0_noscatter  
sample   mass radius length density                               
sample_0 0.3  0.65   0.15   1.506792           NaN           NaN  
                            1.506792           NaN           NaN  
sample_1 0.4  0.65   0.15   2.009056 -4.384631e-09 -8.180859e-09  
sample_2 0.3  0.65   0.15   1.506792           NaN           NaN  
                            1.506792           NaN           NaN

In [56]:
S.mass_table.head()

mass_percent  mass_frac
sample   mass radius length density  element                         
sample_0 0.3  0.65   0.15   1.506792 H                0.75     0.0075
                                     C                2.00     0.0200
                                     N                1.00     0.0100
                                     O               47.00     0.4700
                                     Na               1.22     0.0122

In [58]:
S.sigma_table.head()

mass_percent  mass_frac  \
sample   mass radius length density  element                            
sample_0 0.3  0.65   0.15   1.506792 H                0.75     0.0075   
                                     C                2.00     0.0200   
                                     N                1.00     0.0100   
                                     O               47.00     0.4700   
                                     Na               1.22     0.0122   

                                              atomic_mass  sigma_a  sigma_s  \
sample   mass radius length density  element                                  
sample_0 0.3  0.65   0.15   1.506792 H           1.007940  0.33260   82.020   
                                     C          12.011000  0.00350    5.551   
                                     N          14.006740  1.90000   11.510   
                                     O          15.999400  0.00019    4.232   
                                     Na         22.989768  0.53000    3.280   

                                               Sigma_a   Sigma_s  
sample   mass radius length density  element                      
sample_0 0.3  0.65   0.15   1.506792 H        0.002246  0.553784  
                                     C        0.000005  0.008387  
                                     N        0.001231  0.007456  
                                     O        0.000005  0.112806  
                                     Na       0.000255  0.001579

In [59]:
S.sigma_table_tot

,,,,,mass_frac,Sigma_a,Sigma_s,Sigma_t
sample,mass,radius,length,density,,,,
sample_0,0.3,0.65,0.15,1.506792,0.99876,0.010283,0.718763,0.729046
sample_1,0.4,0.65,0.15,2.009056,0.99876,0.013711,0.958350,0.972061
sample_2,0.3,0.65,0.15,1.506792,1.00126,0.011032,0.903357,0.914389


In [61]:
S.output_scatter

,,,,,mass_frac,Sigma_a,Sigma_s,Sigma_t,x,f_slab,f_cyl,f0,f
sample,mass,radius,length,density,,,,,,,,,
sample_0,0.3,0.65,0.15,1.506792,0.99876,0.010283,0.718763,0.729046,0.088852,0.850142,0.908769,0.861135,0.997731
sample_1,0.4,0.65,0.15,2.009056,0.99876,0.013711,0.958350,0.972061,0.118470,0.816646,0.886428,0.829730,0.997114
sample_2,0.3,0.65,0.15,1.506792,1.00126,0.011032,0.903357,0.914389,0.111441,0.824247,0.891449,0.836847,0.997653


In [62]:
S.shielding_ratio('sample_0')

sample_0/sample                 uncert  \
                                           f_scatter f0_noscatter f_scatter   
sample   mass radius length density                                           
sample_0 0.3  0.65   0.15   1.506792        1.000000     1.000000  0.000000   
sample_1 0.4  0.65   0.15   2.009056        1.000619     1.001205  0.000126   
sample_2 0.3  0.65   0.15   1.506792        1.000077     1.000267  0.000016   

                                                   
                                     f0_noscatter  
sample   mass radius length density                
sample_0 0.3  0.65   0.15   1.506792     0.000000  
sample_1 0.4  0.65   0.15   2.009056     0.000245  
sample_2 0.3  0.65   0.15   1.506792     0.000054

In [14]:
# normalized values by hand

In [15]:
std = S.shielding_table.query('sample=="sample_0"').iloc[0]
ratio = std / S.shielding_table
ratio

,,,,,f_scatter,f0_noscatter
sample,mass,radius,length,density,,
sample_0,0.3,0.65,0.15,1.506792,1.000000,1.000000
sample_1,0.4,0.65,0.15,2.009056,1.000619,1.001205
sample_2,0.3,0.65,0.15,1.506792,1.000077,1.000267


In [16]:
# uncert
uncert = np.abs(std - S.shielding_table) / 2 / np.sqrt(6)
uncert

,,,,,f_scatter,f0_noscatter
sample,mass,radius,length,density,,
sample_0,0.3,0.65,0.15,1.506792,0.000000,0.000000
sample_1,0.4,0.65,0.15,2.009056,0.000126,0.000245
sample_2,0.3,0.65,0.15,1.506792,0.000016,0.000054


# photon attenuation

In [18]:
import neutron_analysis.photonatten as photonatten
import pandas as pd

In [19]:
reload(shielding)
reload(photonatten)

<module 'neutron_analysis.photonatten' from '/Users/wpk/Documents/Research/CurrentResearch/Projects/CCI/collaboration_646/Nick_sharp_workflow/neutron_analysis/neutron_analysis/photonatten.py'>

In [20]:
df_mass = (
    shielding.read_mass_excel('./example_input_data.xlsx', sheet_name='mass', extra_index=['sample','mass'])
    
)

df_mass_limit = (
    df_mass
    .assign(track=lambda x: ~x['track'].isnull())
    .query('track')
    .drop('track', axis=1)
    
)

df_gamma = (
    pd.read_excel('./example_input_data.xlsx', 'gamma')
    .rename(columns={'element': 'element_tot'})
    .assign(element=lambda x: x['element_tot'].str.split('-').str.get(0))
    # energy should be in MeV
    # convert kev to mev
    .assign(energy=lambda x: x['gamma_kev'] / 1000.)
    # using mev in original sheed
    # NOTE: values of these are different
#     .assign(energy=lambda x: x['gamma_mev_used'])
    .set_index('element')

)

df_gamma_limit = (
    df_gamma
    .assign(energy=lambda x: x['gamma_mev_used'])
    
)


S = shielding.NeutronShieldingUser(df_mass)

p = photonatten.PhotonAtten(df_gamma, df_mass)
p_limit = photonatten.PhotonAtten(df_gamma_limit, df_mass_limit)

In [45]:
s_mass = """
sample,mass,radius,length,density,element,mass_frac,track
std,0.10707407371607613,0.65,0.15,0.5377946543242447,As,4.283715682920447e-05,
std,0.10707407371607613,0.65,0.15,0.5377946543242447,Cd,4.888071945425786e-05,
std,0.10707407371607613,0.65,0.15,0.5377946543242447,W,3.5048011360010176e-05,
std,0.10707407371607613,0.65,0.15,0.5377946543242447,Au,9.131046049060264e-06,
std,0.10707407371607613,0.65,0.15,0.5377946543242447,Sm,6.7767795527705954e-06,
std,0.10707407371607613,0.65,0.15,0.5377946543242447,Lu,1.0534318534812894e-05,
std,0.10707407371607613,0.65,0.15,0.5377946543242447,La,8.15185149320727e-06,
std,0.10707407371607613,0.65,0.15,0.5377946543242447,Mo,0.0002650862672008226,
std,0.10707407371607613,0.65,0.15,0.5377946543242447,Zn,0.00026535265296503323,
std,0.10707407371607613,0.65,0.15,0.5377946543242447,C,0.4441525231038234,x
std,0.10707407371607613,0.65,0.15,0.5377946543242447,H,0.06211699778637823,x
std,0.10707407371607613,0.65,0.15,0.5377946543242447,O,0.49303868030635917,x
sample,0.32,0.65,0.15,1.6072451846755507,Si,0.34,x
sample,0.32,0.65,0.15,1.6072451846755507,Na,0.02,x
sample,0.32,0.65,0.15,1.6072451846755507,Ca,0.3,x
sample,0.32,0.65,0.15,1.6072451846755507,Mg,0.02,x
sample,0.32,0.65,0.15,1.6072451846755507,Al,0.07,x
sample,0.32,0.65,0.15,1.6072451846755507,K,0.02,
sample,0.32,0.65,0.15,1.6072451846755507,S,0.01,
sample,0.32,0.65,0.15,1.6072451846755507,Fe,0.006999999999999999,
sample,0.32,0.65,0.15,1.6072451846755507,Ti,0.01,
sample,0.32,0.65,0.15,1.6072451846755507,N,0.01,
sample,0.32,0.65,0.15,1.6072451846755507,O,0.47,x
sample,0.32,0.65,0.15,1.6072451846755507,C,0.005,
sample,0.32,0.65,0.15,1.6072451846755507,Ar,0.001,
sample,0.32,0.65,0.15,1.6072451846755507,H,0.0075,
QC,0.3,0.65,0.15,1.5067923606333287,P,0.000688,
QC,0.3,0.65,0.15,1.5067923606333287,Ar,0.002,
QC,0.3,0.65,0.15,1.5067923606333287,Ti,0.00336,
QC,0.3,0.65,0.15,1.5067923606333287,Mg,0.0146,
QC,0.3,0.65,0.15,1.5067923606333287,H,0.0075,x
QC,0.3,0.65,0.15,1.5067923606333287,Na,0.012199999999999999,x
QC,0.3,0.65,0.15,1.5067923606333287,Ca,0.0191,x
QC,0.3,0.65,0.15,1.5067923606333287,C,0.02,x
QC,0.3,0.65,0.15,1.5067923606333287,K,0.021099999999999997,x
QC,0.3,0.65,0.15,1.5067923606333287,Fe,0.0336,x
QC,0.3,0.65,0.15,1.5067923606333287,Al,0.0737,x
QC,0.3,0.65,0.15,1.5067923606333287,N,0.01,x
QC,0.3,0.65,0.15,1.5067923606333287,O,0.47,x
QC,0.3,0.65,0.15,1.5067923606333287,Si,0.303,x
"""

s_gamma = """
element,element_tot,gamma_kev,gamma_mev_used,energy
Mg,Mg-27,843.0,0.843,0.843
Al,Al-28,1778.8,1.778,1.7788
Ca,Ca-49,3084.5,3.084,3.0845
Cl,Cl-38,1642.59,1.643,1.64259
Mn,Mn-56,1810.7,1.817,1.8107
Na,Na-24,1368.55,1.36855,1.36855
Ti,Ti-51,320.0,0.32008,0.32
V,V-52,1434.0,1.434,1.434
Sc,Sc-46,889.6,0.8896,0.8896000000000001
Tb,Tb-160,879.8,0.8798,0.8797999999999999
Sr,Sr-85,514.5,0.5145,0.5145
Co,Co-60,1332.6,1.3326,1.3326
Eu,Eu-152,1408.0,1.408,1.408
Rb,Rb-86,1077.2,1.0772,1.0772
Ba,Ba-131,496.26,0.49626,0.49626
Cs,Cs-134,796.2,0.7962,0.7962
"""

s_shielding_ratio = """
,,,,,values,values,std/sample,std/sample,uncert,uncert
,,,,,f_scatter,f0_noscatter,f_scatter,f0_noscatter,f_scatter,f0_noscatter
sample,mass,radius,length,density,,,,,,
std,0.10707407371607613,0.65,0.15,0.5377946543242447,0.9986338947598542,0.9969690868084721,1.0,1.0,0.0,0.0
sample,0.32,0.65,0.15,1.6072451846755507,0.9972433607245004,0.9949779466584113,1.0013943778320507,1.0020011902341635,0.00028384157138250897,0.00040643978116812924
QC,0.3,0.65,0.15,1.5067923606333287,0.9980006912125915,0.9962586347315935,1.0006344720528133,1.0007131201197268,0.00012925213284282686,0.0001450204229210921
"""

s_atten_ratio = """
,,,,,,,values,std/sample,uncert
,,,,,,,self_atten_coef,self_atten_coef,self_atten_coef
sample,mass,radius,length,density,element,energy,,,
std,0.10707407371607613,0.65,0.15,0.5377946543242447,Mg,0.843,0.9970407561328006,1.0,0.0
std,0.10707407371607613,0.65,0.15,0.5377946543242447,Al,1.7788,0.9979603981087514,1.0,0.0
std,0.10707407371607613,0.65,0.15,0.5377946543242447,Ca,3.0845,0.9984921583758402,1.0,0.0
std,0.10707407371607613,0.65,0.15,0.5377946543242447,Cl,1.64259,0.9978750485244645,1.0,0.0
std,0.10707407371607613,0.65,0.15,0.5377946543242447,Mn,1.8107,0.9979803881491809,1.0,0.0
std,0.10707407371607613,0.65,0.15,0.5377946543242447,Na,1.36855,0.9976708843212041,1.0,0.0
std,0.10707407371607613,0.65,0.15,0.5377946543242447,Ti,0.32,0.995538738356402,1.0,0.0
std,0.10707407371607613,0.65,0.15,0.5377946543242447,V,1.434,0.9977280555547218,1.0,0.0
std,0.10707407371607613,0.65,0.15,0.5377946543242447,Sc,0.8896000000000001,0.9971117957125745,1.0,0.0
std,0.10707407371607613,0.65,0.15,0.5377946543242447,Tb,0.8797999999999999,0.997096855497794,1.0,0.0
std,0.10707407371607613,0.65,0.15,0.5377946543242447,Sr,0.5145,0.9963170038141316,1.0,0.0
std,0.10707407371607613,0.65,0.15,0.5377946543242447,Co,1.3326,0.9976394834937468,1.0,0.0
std,0.10707407371607613,0.65,0.15,0.5377946543242447,Eu,1.408,0.997705343769972,1.0,0.0
std,0.10707407371607613,0.65,0.15,0.5377946543242447,Rb,1.0772,0.9973688037034297,1.0,0.0
std,0.10707407371607613,0.65,0.15,0.5377946543242447,Ba,0.49626,0.996263088537557,1.0,0.0
std,0.10707407371607613,0.65,0.15,0.5377946543242447,Cs,0.7962,0.9969672556709652,1.0,0.0
sample,0.32,0.65,0.15,1.6072451846755507,Mg,0.843,0.9892651992123584,1.007859931721679,0.001604401844269066
sample,0.32,0.65,0.15,1.6072451846755507,Al,1.7788,0.992565004913319,1.0054358084042099,0.0011095797441538903
sample,0.32,0.65,0.15,1.6072451846755507,Ca,3.0845,0.9943608616230237,1.0041547258266714,0.0008480798580423345
sample,0.32,0.65,0.15,1.6072451846755507,Cl,1.64259,0.9922677680218592,1.005650975153394,0.0011535004729134265
sample,0.32,0.65,0.15,1.6072451846755507,Mn,1.8107,0.9926346341382518,1.0053854195965768,0.0010992941718665308
sample,0.32,0.65,0.15,1.6072451846755507,Na,1.36855,0.9915458733974586,1.0061772340423933,0.0012609226188011302
sample,0.32,0.65,0.15,1.6072451846755507,Ti,0.32,0.9836387882940779,1.0120978861386325,0.002469470667161591
sample,0.32,0.65,0.15,1.6072451846755507,V,1.434,0.9917504167576338,1.006027362021819,0.0012303301207072042
sample,0.32,0.65,0.15,1.6072451846755507,Sc,0.8896000000000001,0.9895236391538149,1.0076684944740164,0.0015653248797243896
sample,0.32,0.65,0.15,1.6072451846755507,Tb,0.8797999999999999,0.9894692816354632,1.0077087525645299,0.0015735425280391162
sample,0.32,0.65,0.15,1.6072451846755507,Sr,0.5145,0.9866122870727403,1.0098364036902328,0.0020078474954249776
sample,0.32,0.65,0.15,1.6072451846755507,Co,1.3326,0.9914335469619433,1.006259558747856,0.001277727078935165
sample,0.32,0.65,0.15,1.6072451846755507,Eu,1.408,0.9916691552025819,1.0060868975663129,0.0012424827628379378
sample,0.32,0.65,0.15,1.6072451846755507,Rb,1.0772,0.9904570361456753,1.0069783618122916,0.0014244521400531777
sample,0.32,0.65,0.15,1.6072451846755507,Ba,0.49626,0.9864127435157788,1.0099860277419668,0.0020383893770913233
sample,0.32,0.65,0.15,1.6072451846755507,Cs,0.7962,0.9889976773614517,1.008058237639926,0.0016448808703256883
QC,0.3,0.65,0.15,1.5067923606333287,Mg,0.843,0.9922680749126975,1.0048098707806588,0.0009818107617780114
QC,0.3,0.65,0.15,1.5067923606333287,Al,1.7788,0.9946525318893829,1.0033256500268342,0.0006788454690680729
QC,0.3,0.65,0.15,1.5067923606333287,Ca,3.0845,0.9959725451656821,1.0025298018729412,0.0005163936449202541
QC,0.3,0.65,0.15,1.5067923606333287,Cl,1.64259,0.9944361070021313,1.003458182479617,0.000705898542707848
QC,0.3,0.65,0.15,1.5067923606333287,Mn,1.8107,0.9947032270754269,1.0032946118847823,0.000672509834852151
QC,0.3,0.65,0.15,1.5067923606333287,Na,1.36855,0.9939122853683504,1.0037816203785637,0.0007719200273659617
QC,0.3,0.65,0.15,1.5067923606333287,Ti,0.32,0.9882587475161448,1.0073664825720536,0.0015036769583863685
QC,0.3,0.65,0.15,1.5067923606333287,V,1.434,0.9940603119356215,1.003689659042879,0.0007531484983249404
QC,0.3,0.65,0.15,1.5067923606333287,Sc,0.8896000000000001,0.9924540217496061,1.004693188662541,0.0009579931241533697
QC,0.3,0.65,0.15,1.5067923606333287,Tb,0.8797999999999999,0.9924149131873384,1.0047177266768579,0.0009630019253514957
QC,0.3,0.65,0.15,1.5067923606333287,Sr,0.5145,0.9903643413664722,1.006010578328624,0.0012269041636799355
QC,0.3,0.65,0.15,1.5067923606333287,Co,1.3326,0.9938309907036821,1.003832133255744,0.0007822309252436136
QC,0.3,0.65,0.15,1.5067923606333287,Eu,1.408,0.9940015048875621,1.0037261904174168,0.0007606054339266269
QC,0.3,0.65,0.15,1.5067923606333287,Rb,1.0772,0.9931261004191065,1.0042720690580307,0.0008720324448422956
QC,0.3,0.65,0.15,1.5067923606333287,Ba,0.49626,0.990221612848735,1.0061011349484097,0.001245388956288793
QC,0.3,0.65,0.15,1.5067923606333287,Cs,0.7962,0.9920756238637968,1.004930704564756,0.0010064758546719959
"""

def get_df_mass():
    with StringIO(s_mass) as f:
        df_mass = pd.read_csv(f,index_col=list(range(0, 6)))

    return df_mass

def get_df_gamma():
    with StringIO(s_gamma) as f:
        df_mass = pd.read_csv(f,index_col=0)

    return df_mass


def get_df_shielding_ratio():
    with StringIO(s_shielding_ratio) as f:
        df_ratio = pd.read_csv(f, index_col=list(range(0,5)), header=[0,1])
    return df_ratio

def get_df_atten_ratio():
    with StringIO(s_atten_ratio) as f:
        df_ratio = pd.read_csv(f, index_col=list(range(0,7)), header=[0,1])
    return df_ratio


import neutron_analysis.shielding as shielding
import neutron_analysis.photonatten as photonatten

def test_shielding():

    df_mass = get_df_mass()
    df_ratio = get_df_shielding_ratio()

    S = shielding.NeutronShieldingUser(df_mass)
    S0 = shielding.NeutronShieldingUser(df_mass)


    for s in [S, S0]:
        pd.testing.assert_frame_equal(s.shielding_ratio('std', value_name='values'), df_ratio)


def test_atten():
    df_mass = get_df_mass()
    df_gamma = get_df_gamma()

    df_ratio = get_df_atten_ratio()

    p = photonatten.PhotonAtten(df_gamma, df_mass)

    pd.testing.assert_frame_equal(p.atten_ratio(value_name='values'), df_ratio)




In [46]:
a = get_df_atten_ratio()

In [47]:
g = get_df_gamma()
m = get_df_mass()

In [49]:
m

mass_frac track
sample mass     radius length density  element                 
std    0.107074 0.65   0.15   0.537795 As        0.000043   NaN
                                       Cd        0.000049   NaN
                                       W         0.000035   NaN
                                       Au        0.000009   NaN
                                       Sm        0.000007   NaN
                                       Lu        0.000011   NaN
                                       La        0.000008   NaN
                                       Mo        0.000265   NaN
                                       Zn        0.000265   NaN
                                       C         0.444153     x
                                       H         0.062117     x
                                       O         0.493039     x
sample 0.320000 0.65   0.15   1.607245 Si        0.340000     x
                                       Na        0.020000     x
                                       Ca        0.300000     x
                                       Mg        0.020000     x
                                       Al        0.070000     x
                                       K         0.020000   NaN
                                       S         0.010000   NaN
                                       Fe        0.007000   NaN
                                       Ti        0.010000   NaN
                                       N         0.010000   NaN
                                       O         0.470000     x
                                       C         0.005000   NaN
                                       Ar        0.001000   NaN
                                       H         0.007500   NaN
QC     0.300000 0.65   0.15   1.506792 P         0.000688   NaN
                                       Ar        0.002000   NaN
                                       Ti        0.003360   NaN
                                       Mg        0.014600   NaN
                                       H         0.007500     x
                                       Na        0.012200     x
                                       Ca        0.019100     x
                                       C         0.020000     x
                                       K         0.021100     x
                                       Fe        0.033600     x
                                       Al        0.073700     x
                                       N         0.010000     x
                                       O         0.470000     x
                                       Si        0.303000     x

In [35]:
df_gamma

,element_tot,gamma_kev,gamma_mev_used,energy
element,,,,
Mg,Mg-27,843.00,0.84300,0.84300
Al,Al-28,1778.80,1.77800,1.77880
Ca,Ca-49,3084.50,3.08400,3.08450
Cl,Cl-38,1642.59,1.64300,1.64259
Mn,Mn-56,1810.70,1.81700,1.81070
Na,Na-24,1368.55,1.36855,1.36855
Ti,Ti-51,320.00,0.32008,0.32000
V,V-52,1434.00,1.43400,1.43400
Sc,Sc-46,889.60,0.88960,0.88960


In [34]:
S.shielding_ratio(value_name='values')

values              std/sample  \
                                       f_scatter f0_noscatter  f_scatter   
sample mass     radius length density                                      
std    0.107074 0.65   0.15   0.537795  0.998634     0.996969   1.000000   
sample 0.320000 0.65   0.15   1.607245  0.997243     0.994978   1.001394   
QC     0.300000 0.65   0.15   1.506792  0.998001     0.996259   1.000634   

                                                       uncert               
                                       f0_noscatter f_scatter f0_noscatter  
sample mass     radius length density                                       
std    0.107074 0.65   0.15   0.537795     1.000000  0.000000     0.000000  
sample 0.320000 0.65   0.15   1.607245     1.002001  0.000284     0.000406  
QC     0.300000 0.65   0.15   1.506792     1.000713  0.000129     0.000145

In [33]:
p.atten_ratio(value_name='values')

values  \
                                                       self_atten_coef   
sample mass     radius length density  element energy                    
std    0.107074 0.65   0.15   0.537795 Mg      0.84300        0.997041   
                                       Al      1.77880        0.997960   
                                       Ca      3.08450        0.998492   
                                       Cl      1.64259        0.997875   
                                       Mn      1.81070        0.997980   
                                       Na      1.36855        0.997671   
                                       Ti      0.32000        0.995539   
                                       V       1.43400        0.997728   
                                       Sc      0.88960        0.997112   
                                       Tb      0.87980        0.997097   
                                       Sr      0.51450        0.996317   
                                       Co      1.33260        0.997639   
                                       Eu      1.40800        0.997705   
                                       Rb      1.07720        0.997369   
                                       Ba      0.49626        0.996263   
                                       Cs      0.79620        0.996967   
sample 0.320000 0.65   0.15   1.607245 Mg      0.84300        0.989265   
                                       Al      1.77880        0.992565   
                                       Ca      3.08450        0.994361   
                                       Cl      1.64259        0.992268   
                                       Mn      1.81070        0.992635   
                                       Na      1.36855        0.991546   
                                       Ti      0.32000        0.983639   
                                       V       1.43400        0.991750   
                                       Sc      0.88960        0.989524   
                                       Tb      0.87980        0.989469   
                                       Sr      0.51450        0.986612   
                                       Co      1.33260        0.991434   
                                       Eu      1.40800        0.991669   
                                       Rb      1.07720        0.990457   
                                       Ba      0.49626        0.986413   
                                       Cs      0.79620        0.988998   
QC     0.300000 0.65   0.15   1.506792 Mg      0.84300        0.992268   
                                       Al      1.77880        0.994653   
                                       Ca      3.08450        0.995973   
                                       Cl      1.64259        0.994436   
                                       Mn      1.81070        0.994703   
                                       Na      1.36855        0.993912   
                                       Ti      0.32000        0.988259   
                                       V       1.43400        0.994060   
                                       Sc      0.88960        0.992454   
                                       Tb      0.87980        0.992415   
                                       Sr      0.51450        0.990364   
                                       Co      1.33260        0.993831   
                                       Eu      1.40800        0.994002   
                                       Rb      1.07720        0.993126   
                                       Ba      0.49626        0.990222   
                                       Cs      0.79620        0.992076   

                                                            std/sample  \
                                                       self_atten_coef   
sample mass     radius length density  element energy                    
std    0.107074 0.65   0.15   0.537795 Mg      0.84300        1.000

In [32]:
df_gamma

,element_tot,gamma_kev,gamma_mev_used,energy
element,,,,
Mg,Mg-27,843.00,0.84300,0.84300
Al,Al-28,1778.80,1.77800,1.77880
Ca,Ca-49,3084.50,3.08400,3.08450
Cl,Cl-38,1642.59,1.64300,1.64259
Mn,Mn-56,1810.70,1.81700,1.81070
Na,Na-24,1368.55,1.36855,1.36855
Ti,Ti-51,320.00,0.32008,0.32000
V,V-52,1434.00,1.43400,1.43400
Sc,Sc-46,889.60,0.88960,0.88960


In [31]:
s = StringIO()

df_gamma.to_csv(s)
print(s.getvalue())

element,element_tot,gamma_kev,gamma_mev_used,energy
Mg,Mg-27,843.0,0.843,0.843
Al,Al-28,1778.8,1.778,1.7788
Ca,Ca-49,3084.5,3.084,3.0845
Cl,Cl-38,1642.59,1.643,1.64259
Mn,Mn-56,1810.7,1.817,1.8107
Na,Na-24,1368.55,1.36855,1.36855
Ti,Ti-51,320.0,0.32008,0.32
V,V-52,1434.0,1.434,1.434
Sc,Sc-46,889.6,0.8896,0.8896000000000001
Tb,Tb-160,879.8,0.8798,0.8797999999999999
Sr,Sr-85,514.5,0.5145,0.5145
Co,Co-60,1332.6,1.3326,1.3326
Eu,Eu-152,1408.0,1.408,1.408
Rb,Rb-86,1077.2,1.0772,1.0772
Ba,Ba-131,496.26,0.49626,0.49626
Cs,Cs-134,796.2,0.7962,0.7962



In [21]:
s = StringIO()

df_mass.to_csv(s)

print(s.getvalue())

sample,mass,radius,length,density,element,mass_frac,track
std,0.10707407371607613,0.65,0.15,0.5377946543242447,As,4.283715682920447e-05,
std,0.10707407371607613,0.65,0.15,0.5377946543242447,Cd,4.888071945425786e-05,
std,0.10707407371607613,0.65,0.15,0.5377946543242447,W,3.5048011360010176e-05,
std,0.10707407371607613,0.65,0.15,0.5377946543242447,Au,9.131046049060264e-06,
std,0.10707407371607613,0.65,0.15,0.5377946543242447,Sm,6.7767795527705954e-06,
std,0.10707407371607613,0.65,0.15,0.5377946543242447,Lu,1.0534318534812894e-05,
std,0.10707407371607613,0.65,0.15,0.5377946543242447,La,8.15185149320727e-06,
std,0.10707407371607613,0.65,0.15,0.5377946543242447,Mo,0.0002650862672008226,
std,0.10707407371607613,0.65,0.15,0.5377946543242447,Zn,0.00026535265296503323,
std,0.10707407371607613,0.65,0.15,0.5377946543242447,C,0.4441525231038234,x
std,0.10707407371607613,0.65,0.15,0.5377946543242447,H,0.06211699778637823,x
std,0.10707407371607613,0.65,0.15,0.5377946543242447,O,0.49303868030635917,x

In [79]:
# atten frame
# this interpolates the tables from https://physics.nist.gov/PhysRefData/XrayMassCoef/tab3.html 
# which i scraped
photonatten.get_default_atten_table().query('element=="Al"').head()

,energy,mu_rho,muen_rho,element
438,0.00100,1185.0,1183.0,Al
439,0.00150,402.2,400.1,Al
440,0.00156,362.1,360.0,Al
441,0.00156,3957.0,3829.0,Al
442,0.00200,2263.0,2204.0,Al


In [80]:
# atten frame
# this interpolates the tables from https://physics.nist.gov/PhysRefData/XrayMassCoef/tab3.html 
# which i scraped
photonatten.get_default_atten_table().query('element=="Al"').head().energy.values

array([0.001     , 0.0015    , 0.00155959, 0.00155961, 0.002     ])

In [81]:
p.mass_table.head()

mass_frac track
sample mass     radius length density  element                 
std    0.107074 0.65   0.15   0.537795 As        0.000043   NaN
                                       Cd        0.000049   NaN
                                       W         0.000035   NaN
                                       Au        0.000009   NaN
                                       Sm        0.000007   NaN

In [82]:
p.gamma_table.head()

,element_tot,gamma_kev,gamma_mev_used,energy
element,,,,
Mg,Mg-27,843.00,0.843,0.84300
Al,Al-28,1778.80,1.778,1.77880
Ca,Ca-49,3084.50,3.084,3.08450
Cl,Cl-38,1642.59,1.643,1.64259
Mn,Mn-56,1810.70,1.817,1.81070


In [83]:
p.atten_table.head()

mass_frac  \
sample mass     radius length density  element energy element_other              
std    0.107074 0.65   0.15   0.537795 Mg      0.843  As              0.000043   
                                                      Au              0.000009   
                                                      C               0.444153   
                                                      Cd              0.000049   
                                                      H               0.062117   

                                                                       mu_rho  \
sample mass     radius length density  element energy element_other             
std    0.107074 0.65   0.15   0.537795 Mg      0.843  As             0.062884   
                                                      Au             0.082483   
                                                      C              0.069223   
                                                      Cd             0.064885   
                                                      H              0.137447   

                                                                     muen_rho  
sample mass     radius length density  element energy element_other            
std    0.107074 0.65   0.15   0.537795 Mg      0.843  As             0.025992  
                                                      Au             0.042331  
                                                      C              0.028650  
                                                      Cd             0.027987  
                                                      H              0.056997

In [84]:
p.atten_table_tot.head()

atten_coef  \
sample mass     radius length density  element energy                
std    0.107074 0.65   0.15   0.537795 Mg      0.84300    0.005930   
                                       Al      1.77880    0.004085   
                                       Ca      3.08450    0.003019   
                                       Cl      1.64259    0.004256   
                                       Mn      1.81070    0.004045   

                                                        self_atten_coef  
sample mass     radius length density  element energy                    
std    0.107074 0.65   0.15   0.537795 Mg      0.84300         0.997041  
                                       Al      1.77880         0.997960  
                                       Ca      3.08450         0.998492  
                                       Cl      1.64259         0.997875  
                                       Mn      1.81070         0.997980

In [124]:
std_val = 'std'
#ref_levels = ['element','energy']
ref_levels = ['element','energy']
table = p.atten_table_tot
ref = table.query("sample==@std_val").droplevel(
    [x for x in table.index.names if x not in ref_levels]#, 'energy']]
)

ratio = ref / table

uncert = np.abs(ratio - 1) / 2 / np.sqrt(6)



In [152]:
%timeit -n 1 -r 1 photonatten.reorder_index_levels(ratio, table.index.names)

325 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [142]:
d = {'values': table, 'std/sample': ratio, 'uncert': uncert}

In [143]:
out = (
    pd.concat({k: photonatten.reorder_index_levels(v, table.index.names) for k,v in d.items()}, axis=1)
    .loc[table.index]
)

In [147]:
out

values  \
                                                       atten_coef   
sample mass     radius length density  element energy               
std    0.107074 0.65   0.15   0.537795 Mg      0.84300   0.005930   
                                       Al      1.77880   0.004085   
                                       Ca      3.08450   0.003019   
                                       Cl      1.64259   0.004256   
                                       Mn      1.81070   0.004045   
                                       Na      1.36855   0.004665   
                                       Ti      0.32000   0.008949   
                                       V       1.43400   0.004551   
                                       Sc      0.88960   0.005788   
                                       Tb      0.87980   0.005818   
                                       Sr      0.51450   0.007384   
                                       Co      1.33260   0.004728   
                                       Eu      1.40800   0.004596   
                                       Rb      1.07720   0.005272   
                                       Ba      0.49626   0.007493   
                                       Cs      0.79620   0.006078   
sample 0.320000 0.65   0.15   1.607245 Mg      0.84300   0.021625   
                                       Al      1.77880   0.014944   
                                       Ca      3.08450   0.011321   
                                       Cl      1.64259   0.015545   
                                       Mn      1.81070   0.014804   
                                       Na      1.36855   0.017004   
                                       Ti      0.32000   0.033084   
                                       V       1.43400   0.016591   
                                       Sc      0.88960   0.021100   
                                       Tb      0.87980   0.021211   
                                       Sr      0.51450   0.027017   
                                       Co      1.33260   0.017231   
                                       Eu      1.40800   0.016755   
                                       Rb      1.07720   0.019208   
                                       Ba      0.49626   0.027423   
                                       Cs      0.79620   0.022168   
QC     0.300000 0.65   0.15   1.506792 Mg      0.84300   0.015544   
                                       Al      1.77880   0.010733   
                                       Ca      3.08450   0.008077   
                                       Cl      1.64259   0.011169   
                                       Mn      1.81070   0.010631   
                                       Na      1.36855   0.012225   
                                       Ti      0.32000   0.023668   
                                       V       1.43400   0.011927   
                                       Sc      0.88960   0.015168   
                                       Tb      0.87980   0.015247   
                                       Sr      0.51450   0.019396   
                                       Co      1.33260   0.012389   
                                       Eu      1.40800   0.012045   
                                       Rb      1.07720   0.013811   
                                       Ba      0.49626   0.019685   
                                       Cs      0.79620   0.015933   

                                                                        \
                                                       self_atten_coef   
sample mass     radius length density  element energy                    
std    0.107074 0.65   0.15   0.537795 Mg      0.84300        0.997041   
                                       Al      1.77880        0.997960   
                                       Ca      3.08450        0.998492   
                                       Cl      1.64259        0.997875   
                     

In [146]:
p.atten_ratio(value_name='values')

values  \
                                                       self_atten_coef   
sample mass     radius length density  element energy                    
std    0.107074 0.65   0.15   0.537795 Mg      0.84300        0.997041   
                                       Al      1.77880        0.997960   
                                       Ca      3.08450        0.998492   
                                       Cl      1.64259        0.997875   
                                       Mn      1.81070        0.997980   
                                       Na      1.36855        0.997671   
                                       Ti      0.32000        0.995539   
                                       V       1.43400        0.997728   
                                       Sc      0.88960        0.997112   
                                       Tb      0.87980        0.997097   
                                       Sr      0.51450        0.996317   
                                       Co      1.33260        0.997639   
                                       Eu      1.40800        0.997705   
                                       Rb      1.07720        0.997369   
                                       Ba      0.49626        0.996263   
                                       Cs      0.79620        0.996967   
sample 0.320000 0.65   0.15   1.607245 Mg      0.84300        0.989265   
                                       Al      1.77880        0.992565   
                                       Ca      3.08450        0.994361   
                                       Cl      1.64259        0.992268   
                                       Mn      1.81070        0.992635   
                                       Na      1.36855        0.991546   
                                       Ti      0.32000        0.983639   
                                       V       1.43400        0.991750   
                                       Sc      0.88960        0.989524   
                                       Tb      0.87980        0.989469   
                                       Sr      0.51450        0.986612   
                                       Co      1.33260        0.991434   
                                       Eu      1.40800        0.991669   
                                       Rb      1.07720        0.990457   
                                       Ba      0.49626        0.986413   
                                       Cs      0.79620        0.988998   
QC     0.300000 0.65   0.15   1.506792 Mg      0.84300        0.992268   
                                       Al      1.77880        0.994653   
                                       Ca      3.08450        0.995973   
                                       Cl      1.64259        0.994436   
                                       Mn      1.81070        0.994703   
                                       Na      1.36855        0.993912   
                                       Ti      0.32000        0.988259   
                                       V       1.43400        0.994060   
                                       Sc      0.88960        0.992454   
                                       Tb      0.87980        0.992415   
                                       Sr      0.51450        0.990364   
                                       Co      1.33260        0.993831   
                                       Eu      1.40800        0.994002   
                                       Rb      1.07720        0.993126   
                                       Ba      0.49626        0.990222   
                                       Cs      0.79620        0.992076   

                                                            std/sample  \
                                                       self_atten_coef   
sample mass     radius length density  element energy                    
std    0.107074 0.65   0.15   0.537795 Mg      0.84300        1.000

In [97]:
d = {'values': table, }

atten_coef  \
sample mass     radius length density  element energy                
std    0.107074 0.65   0.15   0.537795 Mg      0.84300    0.005930   
                                       Al      1.77880    0.004085   
                                       Ca      3.08450    0.003019   
                                       Cl      1.64259    0.004256   
                                       Mn      1.81070    0.004045   
                                       Na      1.36855    0.004665   
                                       Ti      0.32000    0.008949   
                                       V       1.43400    0.004551   
                                       Sc      0.88960    0.005788   
                                       Tb      0.87980    0.005818   
                                       Sr      0.51450    0.007384   
                                       Co      1.33260    0.004728   
                                       Eu      1.40800    0.004596   
                                       Rb      1.07720    0.005272   
                                       Ba      0.49626    0.007493   
                                       Cs      0.79620    0.006078   
sample 0.320000 0.65   0.15   1.607245 Mg      0.84300    0.021625   
                                       Al      1.77880    0.014944   
                                       Ca      3.08450    0.011321   
                                       Cl      1.64259    0.015545   
                                       Mn      1.81070    0.014804   
                                       Na      1.36855    0.017004   
                                       Ti      0.32000    0.033084   
                                       V       1.43400    0.016591   
                                       Sc      0.88960    0.021100   
                                       Tb      0.87980    0.021211   
                                       Sr      0.51450    0.027017   
                                       Co      1.33260    0.017231   
                                       Eu      1.40800    0.016755   
                                       Rb      1.07720    0.019208   
                                       Ba      0.49626    0.027423   
                                       Cs      0.79620    0.022168   
QC     0.300000 0.65   0.15   1.506792 Mg      0.84300    0.015544   
                                       Al      1.77880    0.010733   
                                       Ca      3.08450    0.008077   
                                       Cl      1.64259    0.011169   
                                       Mn      1.81070    0.010631   
                                       Na      1.36855    0.012225   
                                       Ti      0.32000    0.023668   
                                       V       1.43400    0.011927   
                                       Sc      0.88960    0.015168   
                                       Tb      0.87980    0.015247   
                                       Sr      0.51450    0.019396   
                                       Co      1.33260    0.012389   
                                       Eu      1.40800    0.012045   
                                       Rb      1.07720    0.013811   
                                       Ba      0.49626    0.019685   
                                       Cs      0.79620    0.015933   

                                                        self_atten_coef  
sample mass     radius length density  element energy                    
std    0.107074 0.65   0.15   0.537795 Mg      0.84300         0.997041  
                                       Al      1.77880         0.997960  
                                       Ca      3.08450         0.998492  
                                       Cl      1.64259         0.997875  
                                       Mn      1.81070         0.997980  
                                     

In [93]:
p.atten_ratio(exclude_std=True, ref_levels=('element','energy'))

std/sample  \
                                                   self_atten_coef   
element energy  sample mass radius length density                    
Al      1.77880 sample 0.32 0.65   0.15   1.607245        1.005436   
                QC     0.30 0.65   0.15   1.506792        1.003326   
Ba      0.49626 sample 0.32 0.65   0.15   1.607245        1.009986   
                QC     0.30 0.65   0.15   1.506792        1.006101   
Ca      3.08450 sample 0.32 0.65   0.15   1.607245        1.004155   
                QC     0.30 0.65   0.15   1.506792        1.002530   
Cl      1.64259 sample 0.32 0.65   0.15   1.607245        1.005651   
                QC     0.30 0.65   0.15   1.506792        1.003458   
Co      1.33260 sample 0.32 0.65   0.15   1.607245        1.006260   
                QC     0.30 0.65   0.15   1.506792        1.003832   
Cs      0.79620 sample 0.32 0.65   0.15   1.607245        1.008058   
                QC     0.30 0.65   0.15   1.506792        1.004931   
Eu      1.40800 sample 0.32 0.65   0.15   1.607245        1.006087   
                QC     0.30 0.65   0.15   1.506792        1.003726   
Mg      0.84300 sample 0.32 0.65   0.15   1.607245        1.007860   
                QC     0.30 0.65   0.15   1.506792        1.004810   
Mn      1.81070 sample 0.32 0.65   0.15   1.607245        1.005385   
                QC     0.30 0.65   0.15   1.506792        1.003295   
Na      1.36855 sample 0.32 0.65   0.15   1.607245        1.006177   
                QC     0.30 0.65   0.15   1.506792        1.003782   
Rb      1.07720 sample 0.32 0.65   0.15   1.607245        1.006978   
                QC     0.30 0.65   0.15   1.506792        1.004272   
Sc      0.88960 sample 0.32 0.65   0.15   1.607245        1.007668   
                QC     0.30 0.65   0.15   1.506792        1.004693   
Sr      0.51450 sample 0.32 0.65   0.15   1.607245        1.009836   
                QC     0.30 0.65   0.15   1.506792        1.006011   
Tb      0.87980 sample 0.32 0.65   0.15   1.607245        1.007709   
                QC     0.30 0.65   0.15   1.506792        1.004718   
Ti      0.32000 sample 0.32 0.65   0.15   1.607245        1.012098   
                QC     0.30 0.65   0.15   1.506792        1.007366   
V       1.43400 sample 0.32 0.65   0.15   1.607245        1.006027   
                QC     0.30 0.65   0.15   1.506792        1.003690   

                                                            uncert  
                                                   self_atten_coef  
element energy  sample mass radius length density                   
Al      1.77880 sample 0.32 0.65   0.15   1.607245        0.001110  
                QC     0.30 0.65   0.15   1.506792        0.000679  
Ba      0.49626 sample 0.32 0.65   0.15   1.607245        0.002038  
                QC     0.30 0.65   0.15   1.506792        0.001245  
Ca      3.08450 sample 0.32 0.65   0.15   1.607245        0.000848  
                QC     0.30 0.65   0.15   1.506792        0.000516  
Cl      1.64259 sample 0.32 0.65   0.15   1.607245        0.001154  
                QC     0.30 0.65   0.15   1.506792        0.000706  
Co      1.33260 sample 0.32 0.65   0.15   1.607245        0.001278  
                QC     0.30 0.65   0.15   1.506792        0.000782  
Cs      0.79620 sample 0.32 0.65   0.15   1.607245        0.001645  
                QC     0.30 0.65   0.15   1.506792        0.001006  
Eu      1.40800 sample 0.32 0.65   0.15   1.607245        0.001242  
                QC     0.30 0.65   0.15   1.506792        0.000761  
Mg      0.84300 sample 0.32 0.65   0.15   1.607245        0.001604  
                QC     0.30 0.65   0.15   1.506792        0.000982  
Mn      1.81070 sample 0.32 0.65   0.15   1.607245        0.001099  
                QC     0.30 0.65   0.15   1.506792        0.000673  
Na      1.36855 sample 0.32 0.65   0.15   1.607245        0.001261  
                QC     0.30 0.65   0.15   1.506792        0.000772  
Rb      1.07720 sa

In [72]:
p.atten_ratio(exclude_std=True, value_name='values')

values  \
                                                   self_atten_coef   
sample mass radius length density  element energy                    
sample 0.32 0.65   0.15   1.607245 Mg      0.84300        0.989265   
                                   Al      1.77880        0.992565   
                                   Ca      3.08450        0.994361   
                                   Cl      1.64259        0.992268   
                                   Mn      1.81070        0.992635   
                                   Na      1.36855        0.991546   
                                   Ti      0.32000        0.983639   
                                   V       1.43400        0.991750   
                                   Sc      0.88960        0.989524   
                                   Tb      0.87980        0.989469   
                                   Sr      0.51450        0.986612   
                                   Co      1.33260        0.991434   
                                   Eu      1.40800        0.991669   
                                   Rb      1.07720        0.990457   
                                   Ba      0.49626        0.986413   
                                   Cs      0.79620        0.988998   
QC     0.30 0.65   0.15   1.506792 Mg      0.84300        0.992268   
                                   Al      1.77880        0.994653   
                                   Ca      3.08450        0.995973   
                                   Cl      1.64259        0.994436   
                                   Mn      1.81070        0.994703   
                                   Na      1.36855        0.993912   
                                   Ti      0.32000        0.988259   
                                   V       1.43400        0.994060   
                                   Sc      0.88960        0.992454   
                                   Tb      0.87980        0.992415   
                                   Sr      0.51450        0.990364   
                                   Co      1.33260        0.993831   
                                   Eu      1.40800        0.994002   
                                   Rb      1.07720        0.993126   
                                   Ba      0.49626        0.990222   
                                   Cs      0.79620        0.992076   

                                                        std/sample  \
                                                   self_atten_coef   
sample mass radius length density  element energy                    
sample 0.32 0.65   0.15   1.607245 Mg      0.84300        1.007860   
                                   Al      1.77880        1.005436   
                                   Ca      3.08450        1.004155   
                                   Cl      1.64259        1.005651   
                                   Mn      1.81070        1.005385   
                                   Na      1.36855        1.006177   
                                   Ti      0.32000        1.012098   
                                   V       1.43400        1.006027   
                                   Sc      0.88960        1.007668   
                                   Tb      0.87980        1.007709   
                                   Sr      0.51450        1.009836   
                                   Co      1.33260        1.006260   
                                   Eu      1.40800        1.006087   
                                   Rb      1.07720        1.006978   
                                   Ba      0.49626        1.009986   
                                   Cs      0.79620        1.008058   
QC     0.30 0.65   0.15   1.506792 Mg      0.84300        1.004810   
                                   Al      1.77880        1.003326   
                                   Ca      3.08450        1.002530   
                                   Cl      1.64259        1.003458   

## look at limited (compare to excel)

In [45]:
p_limit.atten_table.query('element=="Mg" and sample=="std"')

mass_frac  \
sample mass     radius length density  element energy element_other              
std    0.107074 0.65   0.15   0.537795 Mg      0.843  C               0.444153   
                                                      H               0.062117   
                                                      O               0.493039   

                                                                       mu_rho  \
sample mass     radius length density  element energy element_other             
std    0.107074 0.65   0.15   0.537795 Mg      0.843  C              0.069223   
                                                      H              0.137447   
                                                      O              0.069333   

                                                                     muen_rho  
sample mass     radius length density  element energy element_other            
std    0.107074 0.65   0.15   0.537795 Mg      0.843  C              0.028650  
                                                      H              0.056997  
                                                      O              0.028670

In [46]:
p_limit.atten_table_tot.query('element=="Mg"')

,,,,,,,atten_coef,self_atten_coef
sample,mass,radius,length,density,element,energy,,
std,0.107074,0.65,0.15,0.537795,Mg,0.843,0.005927,0.997043
sample,0.320000,0.65,0.15,1.607245,Mg,0.843,0.020351,0.989893
QC,0.300000,0.65,0.15,1.506792,Mg,0.843,0.015230,0.992423


In [47]:
p_limit.atten_ratio(exclude_std=True)

std/sample  \
                                                   self_atten_coef   
sample mass radius length density  element energy                    
sample 0.32 0.65   0.15   1.607245 Mg      0.84300        1.007222   
                                   Al      1.77800        1.004997   
                                   Ca      3.08400        1.003821   
                                   Cl      1.64300        1.005193   
                                   Mn      1.81700        1.004941   
                                   Na      1.36855        1.005677   
                                   Ti      0.32008        1.011112   
                                   V       1.43400        1.005539   
                                   Sc      0.88960        1.007047   
                                   Tb      0.87980        1.007084   
                                   Sr      0.51450        1.009038   
                                   Co      1.33260        1.005752   
                                   Eu      1.40800        1.005594   
                                   Rb      1.07720        1.006413   
                                   Ba      0.49626        1.009175   
                                   Cs      0.79620        1.007405   
QC     0.30 0.65   0.15   1.506792 Mg      0.84300        1.004654   
                                   Al      1.77800        1.003219   
                                   Ca      3.08400        1.002448   
                                   Cl      1.64300        1.003346   
                                   Mn      1.81700        1.003182   
                                   Na      1.36855        1.003659   
                                   Ti      0.32008        1.007128   
                                   V       1.43400        1.003570   
                                   Sc      0.88960        1.004542   
                                   Tb      0.87980        1.004565   
                                   Sr      0.51450        1.005816   
                                   Co      1.33260        1.003708   
                                   Eu      1.40800        1.003606   
                                   Rb      1.07720        1.004134   
                                   Ba      0.49626        1.005904   
                                   Cs      0.79620        1.004771   

                                                            uncert  
                                                   self_atten_coef  
sample mass radius length density  element energy                   
sample 0.32 0.65   0.15   1.607245 Mg      0.84300        0.001474  
                                   Al      1.77800        0.001020  
                                   Ca      3.08400        0.000780  
                                   Cl      1.64300        0.001060  
                                   Mn      1.81700        0.001008  
                                   Na      1.36855        0.001159  
                                   Ti      0.32008        0.002268  
                                   V       1.43400        0.001131  
                                   Sc      0.88960        0.001438  
                                   Tb      0.87980        0.001446  
                                   Sr      0.51450        0.001845  
                                   Co      1.33260        0.001174  
                                   Eu      1.40800        0.001142  
                                   Rb      1.07720        0.001309  
                                   Ba      0.49626        0.001873  
                                   Cs      0.79620        0.001511  
QC     0.30 0.65   0.15   1.506792 Mg      0.84300        0.000950  
                                   Al      1.77800        0.000657  
                                   Ca      3.08400        0.000500  
                                   Cl      1.64300        0.000683  
                  

In [48]:
# maual ratio
df = p_limit.atten_table_tot
df = utils.loc_levels(df, {'element': df_gamma.index,'sample':['std','sample','QC']})[['self_atten_coef']]
ref = df.query('sample=="std"').droplevel([x for x in df.index.names if x not in ['element']])

ratio = (
    (ref / df.query('sample != "std"'))
#    .assign(uncert=lambda x: np.abs(x['self_atten_coef'] - 1)/2/np.sqrt(6))
)

uncert = np.sqrt(ratio - 1) / 2 / np.sqrt(6)

pd.concat({'ratio': ratio, 'uncert': uncert}, axis=1).head()

ratio  \
                                                  self_atten_coef   
sample mass radius length density  element energy                   
sample 0.32 0.65   0.15   1.607245 Mg      0.843         1.007222   
                                   Al      1.778         1.004997   
                                   Ca      3.084         1.003821   
                                   Cl      1.643         1.005193   
                                   Mn      1.817         1.004941   

                                                           uncert  
                                                  self_atten_coef  
sample mass radius length density  element energy                  
sample 0.32 0.65   0.15   1.607245 Mg      0.843         0.017347  
                                   Al      1.778         0.014430  
                                   Ca      3.084         0.012618  
                                   Cl      1.643         0.014709  
                                   Mn      1.817         0.014348

# write to excel file

In [73]:
with pd.ExcelWriter('./tmp.xlsx') as f:
    S.mass_table.to_excel(f, sheet_name='mass')
    S.sigma_table.to_excel(f, sheet_name='sigma_table')
    S.sigma_table_tot.to_excel(f, sheet_name='sigma_table_tot')
#    S.shielding_table.to_excel(f, sheet_name='shielding')
    S.shielding_ratio(value_name='values').to_excel(f, sheet_name='shielding_ratio')
    
    p.gamma_table.to_excel(f, sheet_name='gamma')
    p.atten_table.to_excel(f, sheet_name='photon_atten')
#     p.atten_table_tot.to_excel(f, sheet_name='photon_atten_tot')
    p.atten_ratio(value_name='values').to_excel(f, sheet_name='photon_ratio')



In [74]:
!open ./tmp.xlsx

In [145]:
!open tmp.xlsx

# some other stuff

In [103]:
out = (
    utils.ratio_table(p.atten_table_tot, ['sample','element'], 'self_atten_coef')
    .assign(uncert=lambda x: np.abs(x['self_atten_coef'] -1 ) /2 / np.sqrt(6))
)
out.unstack('sample').loc[df_gamma.index]

self_atten_coef                   uncert               
sample               QC    sample  std        QC    sample  std
element                                                        
Mg             1.004810  1.007860  1.0  0.000982  0.001604  0.0
Al             1.003326  1.005436  1.0  0.000679  0.001110  0.0
Ca             1.002530  1.004155  1.0  0.000516  0.000848  0.0
Cl             1.003458  1.005651  1.0  0.000706  0.001154  0.0
Mn             1.003295  1.005385  1.0  0.000673  0.001099  0.0
Na             1.003782  1.006177  1.0  0.000772  0.001261  0.0
Ti             1.007366  1.012098  1.0  0.001504  0.002469  0.0
V              1.003690  1.006027  1.0  0.000753  0.001230  0.0
Sc             1.004693  1.007668  1.0  0.000958  0.001565  0.0
Tb             1.004718  1.007709  1.0  0.000963  0.001574  0.0
Sr             1.006011  1.009836  1.0  0.001227  0.002008  0.0
Co             1.003832  1.006260  1.0  0.000782  0.001278  0.0
Eu             1.003726  1.006087  1.0  0.000761  0.001242  0.0
Rb             1.004272  1.006978  1.0  0.000872  0.001424  0.0
Ba             1.006101  1.009986  1.0  0.001245  0.002038  0.0
Cs             1.004931  1.008058  1.0  0.001006  0.001645  0.0

In [100]:
out = (
    utils.ratio_table(p_limit.atten_table_tot, ['sample','element'], 'self_atten_coef')
    .assign(uncert=lambda x: np.abs(x['self_atten_coef'] -1 ) /2 / np.sqrt(6))
)
out.unstack('sample').loc[df_gamma.index]

self_atten_coef                   uncert               
sample               QC    sample  std        QC    sample  std
element                                                        
Mg             1.004654  1.007222  1.0  0.000950  0.001474  0.0
Al             1.003219  1.004997  1.0  0.000657  0.001020  0.0
Ca             1.002448  1.003821  1.0  0.000500  0.000780  0.0
Cl             1.003346  1.005193  1.0  0.000683  0.001060  0.0
Mn             1.003182  1.004941  1.0  0.000650  0.001008  0.0
Na             1.003659  1.005677  1.0  0.000747  0.001159  0.0
Ti             1.007128  1.011112  1.0  0.001455  0.002268  0.0
V              1.003570  1.005539  1.0  0.000729  0.001131  0.0
Sc             1.004542  1.007047  1.0  0.000927  0.001438  0.0
Tb             1.004565  1.007084  1.0  0.000932  0.001446  0.0
Sr             1.005816  1.009038  1.0  0.001187  0.001845  0.0
Co             1.003708  1.005752  1.0  0.000757  0.001174  0.0
Eu             1.003606  1.005594  1.0  0.000736  0.001142  0.0
Rb             1.004134  1.006413  1.0  0.000844  0.001309  0.0
Ba             1.005904  1.009175  1.0  0.001205  0.001873  0.0
Cs             1.004771  1.007405  1.0  0.000974  0.001511  0.0

# using periodic table

In [40]:
import neutron_analysis.shielding as shielding
import periodictable
import pandas as pd
import numpy as np

In [41]:
df_sigma = shielding.get_default_sigma_table()

In [43]:
df_sigma

,atomic_mass,sigma_a,sigma_s
element,,,
H,1.007940,0.332600,82.02
D,2.014102,0.000519,7.64
He,4.002602,0.007470,1.34
Li,6.941000,70.500000,1.37
Be,9.012182,0.007600,7.63
...,...,...,...
Ra,NaN,12.800000,13.00
Ac,NaN,NaN,NaN
Th,232.038100,7.370000,13.36


In [44]:
# check agains periodic table
df_check = (
    df_sigma
    .reset_index()
    .assign(
        atomic_mass2=lambda df: df['element'].apply(lambda x: getattr(periodictable, x).mass),
        sigma_a2=lambda df: df['element'].apply(lambda x: getattr(periodictable, x).neutron.absorption),
        sigma_s2=lambda df: df['element'].apply(lambda x: getattr(periodictable, x).neutron.total),
    )
    .assign(
        dmass=lambda x: np.abs(x['atomic_mass'] - x['atomic_mass2']),
        da=lambda x: np.abs(x['sigma_a'] - x['sigma_a2']),
        ds=lambda x: np.abs(x['sigma_s'] - x['sigma_s2']),
    )
    .set_index('element')
    
)

In [46]:
df_check[['atomic_mass','atomic_mass2','dmass']].dropna().query('dmass > 0')

,atomic_mass,atomic_mass2,dmass
element,,,
D,2.014102,2.014102,2.000000e-09
C,12.011000,12.010700,3.000000e-04
N,14.006740,14.006700,4.000000e-05
Na,22.989768,22.989770,2.000000e-06
Al,26.981539,26.981538,1.000000e-06
P,30.973762,30.973761,1.000000e-06
S,32.066000,32.065000,1.000000e-03
Cl,35.452700,35.453000,3.000000e-04
Ti,47.880000,47.867000,1.300000e-02


In [47]:
df_check[['sigma_a','sigma_a2','da']].dropna().query('da > 0')

,sigma_a,sigma_a2,da
element,,,
Nd,51.0,50.5,0.5


In [48]:
df_check[['sigma_s','sigma_s2','ds']].dropna().query('ds > 0')

,sigma_s,sigma_s2,ds
element,,,
Sm,39.0,39.400,0.400
U,8.9,8.908,0.008


In [249]:
reload(utils)

<module 'neutron_analysis.utils' from '/Users/wpk/Documents/Research/CurrentResearch/Projects/CCI/collaboration_646/Nick_sharp_workflow/neutron_analysis/neutron_analysis/utils.py'>

In [244]:
list(utils.reorder_names([...,'a'], list('abcd')))

['b', 'c', 'd', 'a']

In [238]:
def reorder_names(dims_supplied, dims_all):
    """
    Resolves a supplied list containing an ellispsis representing other items, to
    a generator with the 'realized' list of all items
    """
    if ... in dims_supplied:
        if len(set(dims_all)) != len(dims_all):
            raise ValueError("Cannot use ellipsis with repeated dims")
        if len([d for d in dims_supplied if d == ...]) > 1:
            raise ValueError("More than one ellipsis supplied")
        other_dims = [d for d in dims_all if d not in dims_supplied]
        for d in dims_supplied:
            if d == ...:
                yield from other_dims
            else:
                yield d
    else:
        if set(dims_supplied) ^ set(dims_all):
            raise ValueError(
                f"{dims_supplied} must be a permuted list of {dims_all}, unless `...` is included"
            )
        yield from dims_supplied



In [239]:
list(reorder_names(('d',...,'a'), list('abcd') ))

['d', 'b', 'c', 'a']